# Dataset Lending ETL

In [1]:
# Import dependencies
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
sns.set() 

pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [2]:
# import datasets
# accepted applicants data file 
raw_accepted = pd.read_csv('../Archive/accepted_2007_to_2018Q4.csv') 
# rejected applicants data file
raw_rejected = pd.read_csv('../Archive/rejected_2007_to_2018Q4.csv')

In [3]:
#create data copy 
file_rejected = raw_rejected.copy()
file_rejected.head()

,Amount Requested,Application Date,Loan Title,Risk_Score,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code
0,1000.00000,2007-05-26,Wedding Covered but No Honeymoon,693.00000,10%,481xx,NM,4 years,0.00000
1,1000.00000,2007-05-26,Consolidating Debt,703.00000,10%,010xx,MA,< 1 year,0.00000
2,11000.00000,2007-05-27,Want to consolidate my debt,715.00000,10%,212xx,MD,1 year,0.00000
3,6000.00000,2007-05-27,waksman,698.00000,38.64%,017xx,MA,< 1 year,0.00000
4,1500.00000,2007-05-27,mdrigo,509.00000,9.43%,209xx,MD,< 1 year,0.00000


In [4]:
#create data copy 
file_accepted = raw_accepted.copy()
file_accepted.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.00000,3600.00000,3600.00000,36 months,13.99000,123.03000,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.00000,24700.00000,24700.00000,36 months,11.99000,820.28000,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.00000,20000.00000,20000.00000,60 months,10.78000,432.66000,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.00000,35000.00000,35000.00000,60 months,14.85000,829.90000,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.00000,10400.00000,10400.00000,60 months,22.45000,289.91000,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


# Analysis 1 .Binary classification model to accept or reject loan application

In [5]:
''' 
Only keep those columns that are going to be used for analysis 
 
Application Date - date does not play a role in getting rejected or accepted unless you beleive in Numerology 
Loan Title - For out initial model we have left this out since it did not add a lot of value to the rejected decison. 
Zip Code - We saw people getting accepted and rejected for teh same zipcodes so we think that zipcode does not have a big impact on loan application
Policy Code- This is our target column so we have it but made sure it was all '0'

'''
file_rejected = file_rejected[['Amount Requested', 'Risk_Score',
       'Debt-To-Income Ratio', 'Employment Length']]

In [6]:
file_rejected['Label_target'] = 0
file_rejected.head()

,Amount Requested,Risk_Score,Debt-To-Income Ratio,Employment Length,Label_target
0,1000.00000,693.00000,10%,4 years,0
1,1000.00000,703.00000,10%,< 1 year,0
2,11000.00000,715.00000,10%,1 year,0
3,6000.00000,698.00000,38.64%,< 1 year,0
4,1500.00000,509.00000,9.43%,< 1 year,0


In [7]:
# Extact the same columms from the  accepted applicant df to match the rejected data set. We will combine this dataset at a later set to make a complete data frame that will be used to train teh classification model. 

file_accepted = file_accepted[['loan_amnt', 'fico_range_low', 'fico_range_high', 'dti', 'emp_length']]
file_accepted.head()

,loan_amnt,fico_range_low,fico_range_high,dti,emp_length
0,3600.00000,675.00000,679.00000,5.91000,10+ years
1,24700.00000,715.00000,719.00000,16.06000,10+ years
2,20000.00000,695.00000,699.00000,10.78000,10+ years
3,35000.00000,785.00000,789.00000,17.06000,10+ years
4,10400.00000,695.00000,699.00000,25.37000,3 years


In [8]:
# Calculate the average FICO score for the high and low range of the applicant
file_accepted['average'] = (file_accepted['fico_range_low'] + file_accepted['fico_range_high'])*0.5

# Drop the FICO high and low scores 
file_accepted = file_accepted.drop(columns=['fico_range_low','fico_range_high'], axis=1)

In [9]:
#Create a target label column for accepted df
file_accepted['label_target']=1   
file_accepted.head()

,loan_amnt,dti,emp_length,average,label_target
0,3600.00000,5.91000,10+ years,677.00000,1
1,24700.00000,16.06000,10+ years,717.00000,1
2,20000.00000,10.78000,10+ years,697.00000,1
3,35000.00000,17.06000,10+ years,787.00000,1
4,10400.00000,25.37000,3 years,697.00000,1


In [10]:
# rename columns for accepted and rejected df
file_accepted.rename(columns= {'loan_amnt': 'Loan_Amount', 'dti': 'Debt_income_ratio', 'average': 'Credit_Score', 'label_target': 'Loan_Status', 'emp_length':'Emp_Length'}, inplace=True)

file_rejected.rename(columns= {'Amount Requested': 'Loan_Amount', 'Debt-To-Income Ratio': 'Debt_income_ratio', 'Risk_Score': 'Credit_Score', 'Label_target': 'Loan_Status', 'Employment Length':'Emp_Length'}, inplace=True)

In [11]:
# rearrange columns in rejected df to concat with the accepted df
file_rejected = file_rejected[['Loan_Amount', 'Debt_income_ratio', 'Emp_Length', 'Credit_Score', 'Loan_Status']]
file_rejected.head()

,Loan_Amount,Debt_income_ratio,Emp_Length,Credit_Score,Loan_Status
0,1000.00000,10%,4 years,693.00000,0
1,1000.00000,10%,< 1 year,703.00000,0
2,11000.00000,10%,1 year,715.00000,0
3,6000.00000,38.64%,< 1 year,698.00000,0
4,1500.00000,9.43%,< 1 year,509.00000,0


In [12]:
# remove % from Debt_income_ratio
file_rejected['Debt_income_ratio'] = file_rejected['Debt_income_ratio'].str.replace('%','')
file_rejected['Debt_income_ratio'] = pd.to_numeric(file_rejected['Debt_income_ratio'])
file_rejected.head()


,Loan_Amount,Debt_income_ratio,Emp_Length,Credit_Score,Loan_Status
0,1000.00000,10.00000,4 years,693.00000,0
1,1000.00000,10.00000,< 1 year,703.00000,0
2,11000.00000,10.00000,1 year,715.00000,0
3,6000.00000,38.64000,< 1 year,698.00000,0
4,1500.00000,9.43000,< 1 year,509.00000,0


In [13]:
# Check number of NaNs in rejected dataset

file_rejected.isnull().sum()

Loan_Amount                 0
Debt_income_ratio           0
Emp_Length             951355
Credit_Score         18497630
Loan_Status                 0
dtype: int64

In [14]:
# Check number of NaNs in accepted dataset

file_accepted.isnull().sum()

Loan_Amount              33
Debt_income_ratio      1744
Emp_Length           146940
Credit_Score             33
Loan_Status               0
dtype: int64

In [15]:
# Drop NaNs from both datasets

file_accepted = file_accepted.dropna()
file_rejected = file_rejected.dropna()
print(f'There are {file_accepted.shape[0]} succesfull applications and {file_rejected.shape[0]} unsuccessful applications')

There are 2113648 succesfull applications and 8992595 unsuccessful applications


In [16]:
# Save cleaned data sets 
file_accepted.to_csv('../Archive/accepted_data_clean', index=False)
file_rejected.to_csv('../Archive/rejected_data_clean', index=False)

In [17]:
# Concatinate the two dataframes to single 
consolidated = pd.concat([file_accepted, file_rejected], axis=0).reset_index(drop=True)

In [18]:
# Check unique values in Emp_length col
consolidated['Emp_Length'].value_counts()

< 1 year     8358386
10+ years     960253
5 years       365812
2 years       282099
3 years       248095
1 year        224209
4 years       186245
6 years       137652
7 years       122446
8 years       119819
9 years       101227
Name: Emp_Length, dtype: int64

In [19]:
# Remove the strings years, <,>,+ from Emp_length

consolidated['Emp_Length'] = consolidated['Emp_Length'].replace('10+ years','10')
consolidated['Emp_Length'] = consolidated['Emp_Length'].replace('< 1 year','0')
consolidated['Emp_Length'] = consolidated['Emp_Length'].replace('1 year','1')
consolidated['Emp_Length'] = consolidated['Emp_Length'].str.replace(' years','')
consolidated['Emp_Length'] = consolidated['Emp_Length'].replace(' ','')
consolidated['Emp_Length'] = pd.to_numeric(consolidated['Emp_Length'])
consolidated['Emp_Length'].value_counts()

0     8358386
10     960253
5      365812
2      282099
3      248095
1      224209
4      186245
6      137652
7      122446
8      119819
9      101227
Name: Emp_Length, dtype: int64

In [20]:
# Get dataset summary and identify outliers
consolidated.describe()

,Loan_Amount,Debt_income_ratio,Emp_Length,Credit_Score,Loan_Status
count,11106243.00000,11106243.00000,11106243.00000,11106243.00000,11106243.00000
mean,13132.42598,140.83387,1.55425,641.25526,0.19031
std,10653.05310,15988.09176,3.17176,87.03325,0.39255
min,0.00000,-1.00000,0.00000,0.00000,0.00000
25%,5000.00000,9.89000,0.00000,604.00000,0.00000
50%,10000.00000,19.65000,0.00000,653.00000,0.00000
75%,20000.00000,32.80000,0.00000,687.00000,0.00000
max,1400000.00000,50000031.49000,10.00000,990.00000,1.00000


In [21]:
#counting outliers
consolidated[(consolidated['Debt_income_ratio']>300) | (consolidated['Debt_income_ratio']<0) | (consolidated['Credit_Score']>850) | (consolidated['Credit_Score']<0)].count()

Loan_Amount          161845
Debt_income_ratio    161845
Emp_Length           161845
Credit_Score         161845
Loan_Status          161845
dtype: int64

In [22]:
#Clean up data outliers
consolidated.drop(consolidated[consolidated['Debt_income_ratio']>200].index, inplace = True)
consolidated.drop(consolidated[consolidated['Debt_income_ratio']<0].index, inplace = True)
consolidated.drop(consolidated[consolidated['Credit_Score']<0].index, inplace = True)
consolidated.drop(consolidated[consolidated['Credit_Score']>850].index, inplace = True)
consolidated.shape

(10901852, 5)

### Data Visualization

In [23]:
#Since dataset is very large, we will extract a small sample from our datasaet and plot 
# sample_df = consolidated.sample(frac=0.1, replace=False, random_state=1)

# plt.scatter(sample_df['Debt_income_ratio'],sample_df['Credit_Score'],c=sample_df['Loan_Status'])
# plt.show()

In [24]:
# sample_df['Debt_income_ratio'].plot.density()

In [25]:
# sample_df['Credit_Score'].plot.density()

### Removing Class Imbalance
 

### Method 1: Oversampling
This involves duplicating the minority class. This can potentially lead to overfitting or biasing towards some of minority class outliers/data. 

In [26]:
# Import dependencies
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import ClusterCentroids
from sklearn.preprocessing import StandardScaler
from collections import Counter

In [27]:
# Preprocess the data by using StandardScaler

y = consolidated['Loan_Status']
X = consolidated.drop('Loan_Status',axis=1)
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1, stratify=y)

scaler = StandardScaler()

scaler = scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
print(f' The count for y_train before oversample is:{Counter(y_train)}')

 The count for y_train before oversample is:Counter({0: 6591467, 1: 1584922})


In [28]:
# Oversampling the data
over = RandomOverSampler(sampling_strategy='minority') # 'minority' can be replaced by a float < 1
X_train_over,y_train_over = over.fit_resample(X_train_scaled,y_train)
print(f' The count for y_train after oversample is:{Counter(y_train_over)}')

 The count for y_train after oversample is:Counter({0: 6591467, 1: 6591467})


### Method 2: Undersampling
This involves removing samples from the majority class. This can potentially lead to losing some of teh important infromation from the dataset.

In [30]:
# Undersampling the data
under = RandomUnderSampler(sampling_strategy='majority') # 'majority' can be replaced by a float < 1
X_train_under,y_train_under = under.fit_resample(X_train_scaled,y_train)
print(f' The count for y_train after undersample is:{Counter(y_train_under)}')

 The count for y_train after undersample is:Counter({0: 1584922, 1: 1584922})


### Mehtod 3: Oversampling SMOTE
The synthetic minority oversampling technique (SMOTE) is another oversampling approach where new instances of minority class are interpolated. 

In [31]:
# Resampling using SMOTE the data
smote = SMOTE(sampling_strategy='auto') 
X_train_smote,y_train_smote = smote.fit_resample(X_train_scaled,y_train)
print(f' The count for y_train after SMOTE oversample is:{Counter(y_train_smote)}')

 The count for y_train after SMOTE oversample is:Counter({0: 6591467, 1: 6591467})


### Method 4: Cluster Centroid Umdersampling
The algorithm identifies clusters of the majority class, then generates synthetic data points, called centroids, that are representative of the clusters. The majority class is then undersampled down to the size of the minority class.

In [32]:
# Resampling using Cluster Centroids
cc = ClusterCentroids() 
X_train_cc,y_train_cc = cc.fit_resample(X_train_scaled,y_train)
print(f' The count for y_train after CC undersample is:{Counter(y_train_cc)}')

KeyboardInterrupt: 

### Logistic Regression 

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [34]:
session_names = ['Imbalanced', 'Oversampling', 'Undersampling', 'SMOTE', 'ClusterCentroids']
X_train_sets = [X_train_scaled, X_train_over, X_train_under, X_train_smote, X_train_cc]
y_train_sets = [y_train, y_train_over, y_train_under, y_train_smote, y_train_cc]

In [36]:
# Run Logistic Regression Loop 

lr = LogisticRegression(solver='lbfgs',max_iter=100)

for session in range(len(X_train_sets)):

    lr.fit(X_train_sets[session], y_train_sets[session])
    y_pred = lr.predict(X_test_scaled)

    print(f'This is the result from {session_names[session]} method')
    print(f'accuracy: {accuracy_score(y_test, y_pred):.3f}')
    # confusion_matrix = confusion_matrix(y_test, y_pred)
    # print(confusion_matrix)
    print(classification_report(y_test, y_pred))
    print('------------------------------------')
    print('------------------------------------')



This is the result from Imbalanced method
accuracy: 0.886
              precision    recall  f1-score   support

           0       0.91      0.95      0.93   2197156
           1       0.76      0.61      0.67    528307

    accuracy                           0.89   2725463
   macro avg       0.83      0.78      0.80   2725463
weighted avg       0.88      0.89      0.88   2725463

------------------------------------
------------------------------------
This is the result from Oversampling method
accuracy: 0.877
              precision    recall  f1-score   support

           0       0.96      0.88      0.92   2197156
           1       0.63      0.85      0.73    528307

    accuracy                           0.88   2725463
   macro avg       0.80      0.87      0.82   2725463
weighted avg       0.90      0.88      0.88   2725463

------------------------------------
------------------------------------
This is the result from Undersampling method
accuracy: 0.876
              preci